In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install laserembeddings==1.1.2
#!python -m laserembeddings download-models

from tqdm.auto import tqdm
import re
tqdm.pandas()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from laserembeddings import Laser

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os
#os.system('python -m laserembeddings download-models')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.3 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=bf25c801c5aba29fa629dfdf0a611aa77fde11c6c9d72b74a8c9d87fb85c432e
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.53
    Uninstalling sacremoses-0.0.53:
      Successfully uninstalled sacremoses-0.0.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.
/kaggle/input/assets/en_bn_quran_tafsir.csv
/kaggle/inpu

In [2]:
ls /opt/conda/lib/python3.7/site-packages/laserembeddings/data

In [3]:
import site
loc = site.getsitepackages() 
data_path = loc[0]+'/laserembeddings/data'
print(site.getusersitepackages()) # String for user-specific package location)
loc[0]

/root/.local/lib/python3.7/site-packages


'/opt/conda/lib/python3.7/site-packages'

In [4]:

if not os.path.exists(data_path):
    os.makedirs('data')
import shutil
shutil.copy('../input/assets/93langs.fcodes', data_path)
shutil.copy('../input/assets/93langs.fvocab', data_path)
shutil.copy('../input/assets/bilstm.93langs.2018-12-26.pt', data_path)

!ls '/opt/conda/lib/python3.7/site-packages/laserembeddings/data'

93langs.fcodes	93langs.fvocab	bilstm.93langs.2018-12-26.pt


In [5]:
# shutil.copy('/opt/conda/lib/python3.7/site-packages/laserembeddings/data/93langs.fcodes', './')
# shutil.copy('/opt/conda/lib/python3.7/site-packages/laserembeddings/data/93langs.fvocab', './')
# shutil.copy('/opt/conda/lib/python3.7/site-packages/laserembeddings/data/bilstm.93langs.2018-12-26.pt', './')


In [6]:
corpus_emb_quran = np.load('../input/assets/Holy_Quran_mlt_emb.npy')
corpus_emb_hadith = np.load('../input/assets/en_emb_bukhari_muslim.npy')

In [7]:
en_bn_quran_tafsir = pd.read_csv('../input/assets/en_bn_quran_tafsir.csv')

en_bn_bukhari_muslim = pd.read_csv('../input/assets/en_bn_bukhari_muslim.csv')

In [8]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

def pairwise_euclidean_dists(x, y):
    dists = -2 * np.matmul(x, y.T)
    dists +=  np.sum(x**2, axis=1)[:, np.newaxis]
    dists += np.sum(y**2, axis=1)
    return  np.sqrt(dists)


laser = Laser()

In [9]:
en_bn_bukhari_muslim.head(1)

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated Umar bin Al-Khattab: ...,"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin '...",উমর বিন আল-খাত্তাবের বর্ণনায় উমর বিন আল-খাত্ত...


In [10]:
def MLT_Sahih_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot',query_embedding=None):

    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb_hadith, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_hadith, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]

    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'source'] = str(en_bn_bukhari_muslim['source'][index])
        find.loc[i,'chapter_no'] = str(en_bn_bukhari_muslim['chapter_no'][index])
        find.loc[i,'hadith_no'] = str(en_bn_bukhari_muslim['hadith_no'][index])
        find.loc[i,'chapter'] = str(en_bn_bukhari_muslim['chapter'][index])
        find.loc[i,'text_ar'] = str(en_bn_bukhari_muslim['text_ar'][index])
        find.loc[i,'text_en'] = str(en_bn_bukhari_muslim['text_en'][index])
        find.loc[i,'text_bn'] = str(en_bn_bukhari_muslim['text_bn'][index])
        find.loc[i,'narrators'] = str(en_bn_bukhari_muslim['narrators'][index])
        
    if(metric == 'dot'):
        for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
            find.loc[j,'Score'] = simScore
    else:
        for j,simScore in enumerate(linear_similarities[:-(size+1):]):
            find.loc[j,'Score'] = simScore
        
    return find

In [11]:
en_bn_quran_tafsir.head(1)

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nب...,পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্...


In [12]:
def Multilingual_Quran_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot'):

    query_embedding = laser.embed_sentences(query, lang=language)

    mlt_hadiths = MLT_Sahih_Hadith_Search_Engine(query,size=size,language = 'en',metric = 'dot',query_embedding=query_embedding)
    
    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb_quran, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_quran, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]

    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'Name'] = str(en_bn_quran_tafsir['Name'][index])
        find.loc[i,'Surah'] = str(en_bn_quran_tafsir['Surah'][index])
        find.loc[i,'Ayat'] = str(en_bn_quran_tafsir['Ayat'][index])
        find.loc[i,'Verse'] = str(en_bn_quran_tafsir['Verse'][index]) 
        find.loc[i,'Tafseer'] = str(en_bn_quran_tafsir['Tafseer'][index]) 
        find.loc[i,'ar_text'] = str(en_bn_quran_tafsir['ar_text'][index])
        
        find.loc[i,'আল_বায়ান'] = str(en_bn_quran_tafsir['আল_বায়ান'][index])
        find.loc[i,'tafsir_bayan'] = str(en_bn_quran_tafsir['tafsir_bayan'][index])
        
    if(metric == 'dot'):
        for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
            find.loc[j,'Score'] = simScore
    else:
        for j,simScore in enumerate(linear_similarities[:-(size+1):]):
            find.loc[j,'Score'] = simScore

    return find,mlt_hadiths

In [13]:
%%time
find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("who is near to allah?",size=2,language = 'en',metric = 'l2')

CPU times: user 11.7 s, sys: 293 ms, total: 12 s
Wall time: 11.6 s


In [14]:
find

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Score
0,The Event,56,11,Those are they who will be brought nigh,they are the ones brought near to God,اُولٰٓئِکَ الۡمُقَرَّبُوۡنَ ﴿ۚ۱۱﴾\nاولئک المق...,তারাই সান্নিধ্যপ্রাপ্ত।,(১১) তারাই হবে নৈকট্যপ্রাপ্ত।-তাফসীরে আহসানুল ...,0.211733
1,The Angels,35,17,That is not a hard thing for Allah.,And that is not an arduous thing for God.,وَ مَا ذٰلِکَ عَلَی اللّٰهِ بِعَزِیۡزٍ ﴿۱۷﴾\n...,আর তা আল্লাহর জন্য মোটেই কঠিন নয়।,(১৭) আর আল্লাহর পক্ষে তা কঠিন নয়।-তাফসীরে আহসা...,0.245469
2,The Ascending Stairways,70,29,And those who preserve their chastity,and those who guard their private parts,وَ الَّذِیۡنَ هُمۡ لِفُرُوۡجِهِمۡ حٰفِظُوۡنَ ...,আর যারা তাদের যৌনাংগসমূহের হিফাযতকারী।,(২৯) আর যারা নিজেদের যৌন অঙ্গকে সংযত রাখে।-তাফ...,0.253586
3,The Night,92,5,As for him who giveth and is dutiful (toward A...,As for him who gives what is due to God and is...,فَاَمَّا مَنۡ اَعۡطٰی وَ اتَّقٰی ۙ﴿۵﴾\nفاما م...,"সুতরাং যে দান করেছে এবং তাকওয়া অবলম্বন করেছে,","৫। সুতরাং যে দান করে ও আল্লাহকে ভয় করে, [1][1]...",0.254416
4,The Overthrowing,81,26,Whither then go ye ?,So where are you going? So what path are you g...,فَاَیۡنَ تَذۡهَبُوۡنَ ﴿ؕ۲۶﴾\nفاین تذهبون ﴿۲۶﴾,সুতরাং তোমরা কোথায় যাচ্ছ?,"২৬। সুতরাং তোমরা কোথায় চলেছ? [1][1] অর্থাৎ, কে...",0.258155
...,...,...,...,...,...,...,...,...,...
6228,The Cave,18,93,"Till, when he came between the two mountains, ...",until when he reached between the two barriers...,حَتّٰۤی اِذَا بَلَغَ بَیۡنَ السَّدَّیۡنِ وَجَ...,অবশেষে যখন সে দুই পর্বতের মধ্যবর্তী স্থানে পৌঁ...,(৯৩) চলতে চলতে সে যখন দুই পর্বত-প্রাচীরের[1] ম...,0.910150
6229,The Winnowing Winds,51,49,"And all things We have created by pairs, that ...",And of all things wa-min kulli shay’in is sema...,وَ مِنۡ کُلِّ شَیۡءٍ خَلَقۡنَا زَوۡجَیۡنِ لَع...,আর প্রত্যেক বস্তু থেকে আমি জোড়ায় জোড়ায় সৃষ্টি ...,(৪৯) আমি প্রত্যেক বস্তু সৃষ্টি করেছি জোড়ায়-জোড়...,0.911489
6230,The Believers,23,50,And We made the son of Mary and his mother a p...,And We made the son of Mary Jesus and his moth...,وَ جَعَلۡنَا ابۡنَ مَرۡیَمَ وَ اُمَّهٗۤ اٰیَۃ...,আর আমি মারইয়াম-পুত্র ও তার মাকে নিদর্শন বানালা...,(৫০) এবং আমি মারয়্যাম তনয় (ঈসা) ও তার জননীকে ক...,0.927193
6231,Light,24,11,Lo! they who spread the slander are a gang amo...,Truly those who initiated the slander the wors...,اِنَّ الَّذِیۡنَ جَآءُوۡ بِالۡاِفۡکِ عُصۡبَۃ...,"নিশ্চয়ই যারা এ অপবাদ* রটনা করেছে, তারা তোমাদের...",(১১) যারা মিথ্যা অপবাদ রটনা করেছে[1] তারা তো ত...,0.928979


In [15]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,87,6918,Punishment of Disbelievers at War with Allah a...,حدثنا عبد العزيز بن عبد الله، حدثني إبراهيم بن...,Narrated Ibn `Abbas: I us...,ইবনে আব্বাস বলেনঃ আমি মুহাজিরদের কিছু লোককে কো...,"'Abdul 'Aziz bin 'Abdullah al Uvaisi ,Ibrahim ...",0.003140
1,Sahih Muslim,33,4779,The Book of Jihad and Expedition,حدثنا أبو بكر بن أبي شيبة، حدثنا هاشم بن القاس...,It has been narrated on the authority of Ibn ...,এটি ইবনে সালামার কর্তৃত্বে বর্ণিত হয়েছে তিনি ...,"Abu Bakr bin Abi Shayba ,Hashim bin al Qasim b...",0.002526


In [16]:
find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("আমি কি পুরস্কার পেতে পারি?",language = 'bn',metric = 'l2')
find

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Score
0,The Pen,68,3,And lo! thine verily will be a reward unfailing.,And assuredly you will have an unfailing reward.,وَ اِنَّ لَکَ لَاَجۡرًا غَیۡرَ مَمۡنُوۡنٍ ۚ﴿۳...,আর নিশ্চয় তোমার জন্য রয়েছে অফুরন্ত পুরস্কার।,(৩) তোমার জন্য অবশ্যই রয়েছে নিরবচ্ছিন্ন পুরস্ক...,0.411235
1,The Star,53,59,"Marvel ye then at this statement,",Do you then marvel in denial at this discourse...,اَفَمِنۡ هٰذَا الۡحَدِیۡثِ تَعۡجَبُوۡنَ ﴿ۙ۵۹﴾...,তোমরা কি এ কথায় বিস্ময় বোধ করছ?,(৫৯) তোমরা কি এই কথায় বিস্ময়বোধ করছ? [1][1] এখ...,0.412561
2,The Calamity,101,10,"Ah, what will convey unto thee what she is! -",And what will show you what it is? that is to ...,وَ مَاۤ اَدۡرٰىکَ مَا هِیَهۡ ﴿ؕ۱۰﴾\nو ما ادرى...,আর তোমাকে কিসে জানাবে হাবিয়া কী?,"১০। কিসে তোমাকে জানাল, তা কি? [1][1] এখানে জাহ...",0.415512
3,The Clot,96,10,A slave when he prayeth ?,a servant namely Muhammad (s) when he prays?,عَبۡدًا اِذَا صَلّٰی ﴿ؕ۱۰﴾\nعبدا اذا صلی ﴿۱۰﴾,"এক বান্দাকে, যখন সে সালাত আদায় করে?",১০। এক বান্দা (রসূলুল্লাহ)কে যখন সে নামায আদায়...,0.416029
4,The Overthrowing,81,26,Whither then go ye ?,So where are you going? So what path are you g...,فَاَیۡنَ تَذۡهَبُوۡنَ ﴿ؕ۲۶﴾\nفاین تذهبون ﴿۲۶﴾,সুতরাং তোমরা কোথায় যাচ্ছ?,"২৬। সুতরাং তোমরা কোথায় চলেছ? [1][1] অর্থাৎ, কে...",0.416262
...,...,...,...,...,...,...,...,...,...
6220,Yâ Sîn,36,39,And for the moon We have appointed mansions ti...,And the moon — read wa’l-qamaru in the nominat...,وَ الۡقَمَرَ قَدَّرۡنٰهُ مَنَازِلَ حَتّٰی عَا...,"আর চাঁদের জন্য আমি নির্ধারণ করেছি মানযিলসমূহ, ...",(৩৯) এবং চন্দ্রের জন্য আমি বিভিন্ন কক্ষ নির্দি...,0.882083
6221,The Moon,54,55,Firmly established in the favour of a Mighty K...,in an abode of truth a place of truth wherein ...,فِیۡ مَقۡعَدِ صِدۡقٍ عِنۡدَ مَلِیۡکٍ مُّقۡتَد...,"যথাযোগ্য আসনে, সর্বশক্তিমান মহাঅধিপতির নিকটে।","(৫৫) যথাযোগ্য আসনে,[1] সার্বভৌমক্ষমতার অধিকারী...",0.882390
6222,Iron,57,18,"Lo! those who give alms, both men and women, a...",Indeed men who give voluntary alms al-mussaddi...,اِنَّ الۡمُصَّدِّقِیۡنَ وَ الۡمُصَّدِّقٰتِ وَ...,নিশ্চয় দানশীল পুরুষ ও দানশীল নারী এবং যারা আল্...,(১৮) দানশীল পুরুষগণ ও দানশীল নারিগণ এবং যারা আ...,0.882405
6223,Palm Fibre,111,4,"And his wife, the wood-carrier,",and his wife wa’mra’atuhu is a supplement to t...,وَّ امۡرَاَتُهٗ ؕ حَمَّالَۃَ الۡحَطَبِ ۚ﴿۴﴾\n...,"আর তার স্ত্রী লাকড়ি বহনকারী,",৪। এবং তার স্ত্রীও; যে ইন্ধন বহনকারিণী। [1][1]...,0.888125


In [17]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,87,6918,Punishment of Disbelievers at War with Allah a...,حدثنا عبد العزيز بن عبد الله، حدثني إبراهيم بن...,Narrated Ibn `Abbas: I us...,ইবনে আব্বাস বলেনঃ আমি মুহাজিরদের কিছু লোককে কো...,"'Abdul 'Aziz bin 'Abdullah al Uvaisi ,Ibrahim ...",0.007022
1,Sahih Muslim,51,7196,The Book Pertaining to Repentance and Exhortat...,حدثنا حبان بن موسى، أخبرنا عبد الله بن المبارك...,"Said b. Musayyib, Urwa b. Zubair, Alqama b. W...","Said b মুসাইব, উরুওয়া b জুবাইর, আলকামা b ওয়া...","Hiban bin Musa bin Sawar ,'Abdullah bin Mubara...",0.006431
2,Sahih Bukhari,52,2661,Witnesses,حدثنا أبو الربيع، سليمان بن داود وأفهمني بعضه ...,Narrated Aisha: (the wife...,আয়েশা (রাঃ) এর কাহিনী বর্ণিত হয়েছেঃ (নবীজী (...,"Sulaiman bin Da'ud al 'Atki al Zahrani ,Falayh...",0.006297
3,Sahih Bukhari,64,4193,Military Expeditions led by the Prophet pbuh A...,حدثنا عبد العزيز بن عبد الله، حدثنا إبراهيم بن...,Narrated `Aisha: Whenever...,`া আয়েশা (রাঃ) এর কাহিনী `াঃ রাসূলুল্লাহ (সাঃ...,"'Abdul 'Aziz bin 'Abdullah al Uvaisi ,Ibrahim ...",0.006281
4,Sahih Bukhari,54,2732,Conditions,حدثني عبد الله بن محمد، حدثنا عبد الرزاق، أخبر...,Narrated Al-Miswar bin Makhrama and Marwan: ...,আল-মিসওয়ার বিন মখরমা ও মারওয়ান বর্ণনা করেছেন...,"'Abdullah bin Muhammad al Musandi ,'Abdur Razz...",0.004997
5,Sahih Muslim,33,4779,The Book of Jihad and Expedition,حدثنا أبو بكر بن أبي شيبة، حدثنا هاشم بن القاس...,It has been narrated on the authority of Ibn ...,এটি ইবনে সালামার কর্তৃত্বে বর্ণিত হয়েছে তিনি ...,"Abu Bakr bin Abi Shayba ,Hashim bin al Qasim b...",0.004944
6,Sahih Bukhari,60,3342,Prophets,قال عبدان أخبرنا عبد الله، أخبرنا يونس، عن الز...,Narrated Anas (ra): Abu ...,"আবু দাহার (রাঃ) বলেন, রাসূলুল্লাহ (সাঃ) বলেন, ...","'Abdullah bin 'Uthman ''Aabadan ,'Abdullah bin...",0.004739
7,Sahih Bukhari,60,3364,Prophets,وحدثني عبد الله بن محمد، حدثنا عبد الرزاق، أخب...,Narrated Ibn `Abbas: The ...,ইবনে আব্বাস বর্ণিতঃ প্রথম মহিলা যিনি বেল্ট ব্য...,"'Abdullah bin Muhammad al Musandi ,'Abdur Razz...",0.004442
8,Sahih Bukhari,65,4797,Prophetic Commentary on the Qur'an Tafseer of ...,حدثنا يحيى بن بكير، حدثنا الليث، عن يونس، عن ا...,Narrated Aisha: (The wife...,আয়েশা (আ.) এর কাহিনী বর্ণিত হয়েছে: (নবীজী) ত...,"Yahya bin Yahya bin Bukayr ,al Laith bin Sa'd ...",0.003621
9,Sahih Bukhari,65,4595,Prophetic Commentary on the Qur'an Tafseer of ...,حدثني إبراهيم بن موسى، عن هشام، عن معمر،‏.‏ وح...,Narrated Ibn `Abbas: Abu ...,ইবনে আব্বাস বর্ণনা করেছেন: আবু সুফিয়ান আমাকে ...,"Ibrahim bin Musa al Razi al Fara' ,Hisham bin ...",0.003519
